In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedGroupKFold
import wandb

import torch
from torch import nn
from torchvision import models
import torch.optim as optim

from dataset import get_dataloaders, get_tiles_datasets
from utils import seed_everything
from trainer import Trainer

# Params
Image.MAX_IMAGE_PIXELS = 1e11
CFG = {
    'seed': 42,
    'base_model': 'resnet34',   # resnet18/34/50, efficientnet_v2_s/m/l
    'img_size': 512,
    'batch_size': 32,
    'freeze_epochs': 1,
    'epochs': 10,
    'base_lr': 1e-3,
    'affine_degrees': 0,
    'affine_translate': None,
    'affine_scale': None
}

# Wandb
wandb.login(key='1b0401db7513303bdea77fb070097f9d2850cf3b')
run = wandb.init(project='kaggle-ubc-ocean', config=CFG, tags=['torch', 'tiles', 'baseline'])

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Paths
root = '/media/latlab/MR/projects/kaggle-ubc-ocean'
data_dir = os.path.join(root, 'data')
results_dir = os.path.join(root, 'results')
train_csv = 'train_tiles_2048_p25_p5.csv'
train_image_dir = os.path.join(results_dir, 'train_tiles_2048_p25_p5')

# Seed
seed_everything(CFG['seed'])

# Load data
df = pd.read_csv(os.path.join(results_dir, train_csv))

# Label encoder/decoder
encode = {'HGSC': 0, 'LGSC': 1, 'EC': 2, 'CC': 3, 'MC': 4}
decode = {v: k for k, v in encode.items()}

In [ ]:
def train_model(CFG, train_image_dir, df_train, df_validation, encode):
    # Data loaders
    datasets = get_tiles_datasets(CFG, train_image_dir, df_train, df_validation, encode)
    dataloaders = get_dataloaders(CFG, datasets)

    # Model definition
    model = models.get_model(CFG['base_model'], weights='DEFAULT').to(device)
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, len(encode)).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=CFG['base_lr'], momentum=0.9)
    exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    # Training
    trainer = Trainer(model, dataloaders, loss_fn, optimizer, exp_lr_scheduler, device, wandb_log=True)
    model, _ = trainer.train(num_epochs=CFG['freeze_epochs'])
    trainer.unfreeze()
    model, best_balanced_acc = trainer.train(num_epochs=CFG['epochs'])
    return model, best_balanced_acc

In [ ]:
# Split train/valid
skf = StratifiedGroupKFold(n_splits=1, random_state=CFG['seed'], shuffle=True)
lb = df['label']
train_index, valid_index = skf.split(X=np.zeros(len(lb)), y=lb, groups=df['orig_image_id'])
df_train = df.iloc[train_index]
df_validation = df.iloc[valid_index]

# Model training
model, best_balanced_acc = train_model(CFG, train_image_dir, df_train, df_validation, encode)
torch.save(model.state_dict(), os.path.join(results_dir, 'models', f'ubc-ocean_{run.name}.pt'))
wandb.finish()